In [1]:
import pickle
import numpy as np
import pandas as pd
import json

In [3]:
with open('Run200_list.pkl', 'rb') as handle:
    data = pickle.load(handle)

In [3]:
# Define the range for Tstamp_us
#lower_bound = 1650 * ( 60 * 1e6)
#upper_bound = 1700 * ( 60 * 1e6)

# Filter the DataFrame
#data = data[(data['Tstamp_us'] >= lower_bound) & (data['Tstamp_us'] <= upper_bound)]

# Reset index if needed
#data.reset_index(drop=True, inplace=True)

In [4]:
print(data)

         Brd  Ch   LG   HG     Tstamp_us  event_ids  x_pos  y_pos     z_pos
0          2   0  168  210  1.715398e+15          0 -73.90  74.17  159.8156
1          2   1  159  199  1.715398e+15          0 -26.10  74.17  159.8156
2          2   2  156  222  1.715398e+15          0 -73.90  26.37  159.8156
3          2   3  167  224  1.715398e+15          0 -26.10  26.37  159.8156
4          2   4  154  225  1.715398e+15          0  73.90  74.16  159.8156
...      ...  ..  ...  ...           ...        ...    ...    ...       ...
2103355    0  59  152  197  1.715405e+15      10954 -50.01  50.27   78.3578
2103356    0  60  157  193  1.715405e+15      10954 -77.64  66.22   78.3578
2103357    0  61  159  167  1.715405e+15      10954 -22.37  66.22   78.3578
2103358    0  62  148  146  1.715405e+15      10954 -50.01  82.18   78.3578
2103359    0  63  153  356  1.715405e+15      10954  50.01  82.17   78.3578

[2103360 rows x 9 columns]


In [5]:
LG = np.array(data["LG"])
event_ids = np.array(data["event_ids"])
xpos = np.array(data["x_pos"])
ypos = np.array(data["y_pos"])
zpos = np.array(data["z_pos"])

#LG = LG - 50.39

mask = LG > 0

LG = LG[mask]
event_ids = event_ids[mask]
xpos = xpos[mask]
ypos = ypos[mask]
zpos = zpos[mask]

events = list(set(event_ids))

In [14]:
use_log_scale = True  # Set to True for logarithmic scale, False for linear scale
time_window = 1 # All hits below this time value will be given artificial hit times

data = []

numEvts = 50
if len(events) < 20:
    numEvts = len(x)
    


for i in events[0:numEvts]:
    event_mask_1 = False
    event_mask_2 = False
    
    event = {
        "infoText": "CALI BNL Prototype Event #" + str(i),
        "colorScale": {}
    }
    
    mask = (event_ids == i)

    # Filter out energies less than or equal to 0
    valid_energies = [energy for energy in LG[mask] if energy > 300]

    if valid_energies:
        event_mask_1 = True
        min_energy = min(valid_energies)
        max_energy = max(valid_energies)

        event["colorScale"]["min"] = float(min_energy)
        event["colorScale"]["max"] = float(max_energy)

        if use_log_scale:
            # Logarithmic scaling
            normalized_energy = [np.log(energy / min_energy) / np.log(max_energy / min_energy) if energy > 0 else energy for energy in LG[mask]]
        else:
            # Linear scaling
            normalized_energy = [(energy - min_energy) / (max_energy - min_energy) if energy > 0 else energy for energy in LG[mask]]
    else:
        # If all energies are 0 or less, retain the original values
        normalized_energy = LG[mask]

    objects = []

    for j in range(len(LG[mask])):
        dist = np.sqrt((xpos[mask][j] / 200) ** 2 + (ypos[mask][j] / 200) ** 2 + (zpos[mask][j] / 200) ** 2)
        c = 0.299792
        time = dist / c
        if normalized_energy[j] > 0:
            event_mask_2 = True
            obj = {
                "type": "hit",
                "time": float(time),
                "x": float(xpos[mask][j] / 200),
                "y": float(ypos[mask][j] / 200),
                "z": float(zpos[mask][j] / 200),
                "color": float(normalized_energy[j])
            }

            objects.append(obj)
    if event_mask_1 & event_mask_2:
        event["objects"] = objects
        data.append(event)

eventPath = 'Events.json'
# Write data to JSON file
with open(eventPath, 'w') as json_file:
    json.dump(data, json_file, indent=2)

/Users/spreins/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in long_scalars
/Users/spreins/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in long_scalars
